In [1]:
%load_ext autoreload  
%autoreload 2 

from src.active_learner import ActiveLearner, ActiveLearnerConfig

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEBUG = True
config = ActiveLearnerConfig(debug=DEBUG, sampling_sizes=(1000, 2000, 3000, 4000), strategy="uncertainty_sampling", max_steps=10 if DEBUG else 20000)
config

ActiveLearnerConfig(max_length=66, debug=True, model_name='google/electra-small-discriminator', strategy='uncertainty_sampling', sampling_sizes=(1000, 2000, 3000, 4000), max_steps=10, batch_size=8)

In [3]:
wandb_project_name = "debugging" if config.debug else "subset_active_learning_corrected"

import wandb
%env WANDB_PROJECT=$wandb_project_name
%env WANDB_NOTEBOOK_NAME="./train.ipynb"
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find "./train.ipynb".
wandb: Currently logged in as: jjoozzz. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=debugging
env: WANDB_NOTEBOOK_NAME="./train.ipynb"


True

In [4]:
active_learner = ActiveLearner(config)

No config specified, defaulting to: sst/default
Reusing dataset sst (/Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 312.63it/s]
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-c6c7962a7d90e9c5.arrow
100%|██████████| 9/9 [00:00<00:00, 10.95ba/s]
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-90df394ccb2006db.arrow
100%|██████████| 2/2 [00:00<00:00, 24.44ba/s]
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-4a67cf392607269a.arrow
100%|██████████| 3/3 [00:00<00:00, 16.76ba/s]
Some weights of the model checkpoint at google

In [5]:
active_learner.train()

100%|██████████| 1/1 [00:00<00:00, 197.24ba/s]


Current train size: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1000
}) |  Sampling 1000 new samples | Config strategy is uncertainty_sampling
training_args: TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-06,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=300,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_mod

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 10/10 [00:07<00:00,  1.46it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


...

loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /Users/garylai/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "trans

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 10/10 [00:07<00:00,  1.39it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 10/10 [00:07<00:00,  1.27it/s]

{'train_runtime': 10.4395, 'train_samples_per_second': 7.663, 'train_steps_per_second': 0.958, 'train_loss': 0.6793347358703613, 'epoch': 10.0}


train/epoch,▁
train/global_step,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,10.0
train/global_step,10
train/total_flos,303390046080.0
train/train_loss,0.67933
